# User Class

In [1]:
class User:
    def __init__(self, name):
        self.name = name
        self.properties = []
        
    def add_property(self, property):
        self.properties.append(property)
        
    def get_properties(self):
        return self.properties

# Wikipedia Parser

This parses wikipedia to get the gross median rent per state.

In [2]:
import re
import requests

class Parser:
    
    def __init__(self, url):
        self.url = url
        self.rent_data = {}
        self.rent_data = self.parse_states(self.read_html())

    def read_html(self):
        response = requests.get(self.url)
        return response.text

    def parse_states(self, html):
        states_pattern = r'<h2>.*?id="(.*?)".*?</h2>'
        states = re.findall(states_pattern, html, re.DOTALL)
        rent_data_nested = {}

        for state in states:
            state_pattern = rf'id="{state}".*?id=".*?"'  # match id="STATE_NAME" and all the text after it till the next id which is the next state
            state_html = re.findall(state_pattern, html, re.DOTALL)
            state_name = state.replace('_', ' ').lower()
            if state_html:
                rent_data_nested[state_name] = self.save_rent_data(state_html[0])

        return rent_data_nested

    def save_rent_data(self, html):
        pattern = r'<td><a href="[^"]+" title="[^"]+">(.*?)</a></td>\s*<td>\$([\d,]+)</td>\s*<td>\$([\d,]+)</td>\s*<td>\$([\d,]+)</td>\s*<td>\$([\d,]+)</td>\s*<td>\$([\d,]+)'
        matches = re.findall(pattern, html, re.DOTALL)

        rent_data_nested = {}
        for match in matches:
            county = match[0] 
            prices = [int(price.replace(',', '')) for price in match[1:]]  # Convert prices to integers, removing commas
            county_name = county.replace(' County', '').lower()
            rent_data_nested[county_name] = {
                "studio": prices[0],
                "1 bedroom": prices[1],
                "2 bedroom": prices[2],
                "3 bedroom": prices[3],
                "4 bedroom": prices[4]
            }
        
        return rent_data_nested


# Property Class

In [3]:
class Property:
    def __init__(self, county, state, property_type):
        self.county = county
        self.state = state
        self.property_type = property_type
        self.incomes = []
        self.expenses = []
        self.roi = None
        self.confirm_or_input_rent()
        
    def add_income(self, source, amount=None):
        self.incomes.append({'source': source, 'amount': amount})
        
    def confirm_or_input_rent(self):
        parse_object = Parser('https://en.wikipedia.org/wiki/Cost_of_rent_by_state_and_county_in_the_United_States')
        state_data = parse_object.rent_data.get(self.state.lower())
        if state_data:
            county_data = state_data.get(self.county.lower())
            if county_data:
                rent_amount = county_data.get(self.property_type.lower())
                if rent_amount:
                    print(f"The parsed rent for a {self.property_type} in {self.county}, {self.state} is: ${rent_amount}")
                    user_confirmation = input("Is this correct? (yes/no) ")
                    if user_confirmation.lower() == 'yes':
                        self.incomes.append({'source': 'rent', 'amount': rent_amount})
                    else:
                        self.enter_own_rent_amount()
                else:
                    self.enter_own_rent_amount()
            else:
                print("County data not found. Please enter the rent amount manually.")
                self.enter_own_rent_amount()
        else:
            print("State data not found. Please enter the rent amount manually.")
            self.enter_own_rent_amount()
    
    def enter_own_rent_amount(self):
        amount = float(input("Please enter the rent amount: "))
        self.incomes.append({'source': 'rent', 'amount': amount})
    
    def add_expense(self, source, amount):
        self.expenses.append({'source': source, 'amount': amount})
    
    def calculate_roi(self):
        total_income = sum([income['amount'] for income in self.incomes])
        total_expense = sum([expense['amount'] for expense in self.expenses])
        net_income = total_income - total_expense
        investment = total_expense
        self.roi = (net_income / investment) * 100 if investment != 0 else 0
        return self.roi

In [4]:
users = {}
current_user = None

def print_with_lines(text):
    print('\n' + "-" * 50)
    print(text)
    print("-" * 50)

def add_user(name):
    if name in users:
        print_with_lines(f"User '{name}' already exists.")
    else:
        users[name] = User(name)
        print_with_lines(f"User '{name}' added successfully.")

def select_user(name):
    global current_user
    if name in users:
        current_user = users[name]
        print_with_lines(f"User '{name}' selected.")
    else:
        print_with_lines(f"User '{name}' does not exist. Please add the user first.")

def get_current_user():
    return current_user

def add_property_to_user(user):
    state = input("Enter the state for the property (New Jersey, etc): ")
    county = input("Enter the county for the property (Bergen, etc): ")
    property_type = input("Enter the type of property ('Studio', '1 Bedroom', '2 Bedroom', '3 Bedroom', or '4 Bedroom'): ")
    new_property = Property(county, state, property_type)
    user.add_property(new_property)
    print_with_lines(f"Property in {county}, {state} of type {property_type} added to user {user.name}. Information parsed from Median State Incomes Wikipedia Page.")


def add_income_to_property(property):
    source = input("Enter the source of income (e.g., 'Studio', '1 Bedroom', etc., or other income source): ")
    amount = None
    if source not in ["Studio", "1 Bedroom", "2 Bedroom", "3 Bedroom", "4 Bedroom"]:
        amount = float(input("Enter the amount of income: "))
    property.add_income(source, amount)
    print_with_lines(f"Income from {source} added to property in {property.county}, {property.state}.")


def add_expense_to_property(property):
    source = input("Enter the source of expense (e.g., 'Taxes', 'Mortgage', 'Insurance', etc.): ")
    amount = float(input("Enter the amount of expense: "))
    property.add_expense(source, amount)
    print_with_lines(f"Expense for {source} added to property in {property.county}, {property.state}.")


def calculate_roi_for_property(property):
    roi = property.calculate_roi()
    print_with_lines(f"The ROI for the property in {property.county}, {property.state} is: {roi:.2f}%")


def list_properties(user):
    if user.get_properties():
        print_with_lines(f"Properties of user {user.name}:")
        for i, property in enumerate(user.get_properties(), 1):
            print_with_lines(f"{i}. {property.county}, {property.state}")
    else:
        print_with_lines(f"User {user.name} has no properties.")

def print_all_users(avaliable_users):
    avaliable_users = list(users.keys())
    print_with_lines("Avaliable users:")
    for user in avaliable_users:
        print(user)

def main_menu():
    global current_user
    while True:
        print("\nWelcome to the Rental Property ROI Calculator!")
        print("1. Add a new user")
        print("2. Select an existing user")
        print("3. Add a new property")
        print("4. Add income to a property")
        print("5. Add expense to a property")
        print("6. Calculate ROI for a property")
        print("7. List all properties")
        print("8. Exit")

        choice = input("Please enter your choice: ")

        if choice == "1":
            name = input("Enter the name of the new user: ")
            add_user(name)
        elif choice == "2":
            
            # get all users
            avaliable_users = list(users.keys())
            if avaliable_users:
                print_all_users(avaliable_users)
                name = input("Enter the name of the user: ")
                select_user(name)
            else:
                #user array is empty
                print_with_lines("No users found. Please add a user first.")
                
        elif choice == "3":
            if current_user:
                add_property_to_user(current_user)
            else:
                avaliable_users = list(users.keys())
                if avaliable_users:
                    print_with_lines("Please select a user first.")
                    print_all_users(avaliable_users)
                else:
                    print('No users found. Please add a user first.')

        elif choice == "4":
            if current_user:
                property_index = int(input("Enter the number of the property to add income to: ")) - 1
                if 0 <= property_index < len(current_user.get_properties()):
                    add_income_to_property(current_user.get_properties()[property_index])
                else:
                    print_with_lines("Invalid property number.")
            else:
                print_with_lines("Please select a user and a property first.")
        elif choice == "5":
            if current_user:
                property_index = int(input("Enter the number of the property to add expense to: ")) - 1
                if (0 <= property_index) and (property_index < len(current_user.get_properties())):
                    add_expense_to_property(current_user.get_properties()[property_index])
                else:
                    print("Invalid property number.")
            else:
                print("Please select a user and a property first.")
        elif choice == "6":
            if current_user:
                property_index = int(input("Enter the number of the property to calculate ROI for: ")) - 1
                avaliable_properties = current_user.get_properties()
                if avaliable_properties:
                    print_with_lines("Avaliable properties:")
                    for i, property in enumerate(avaliable_properties, 1):
                        print(f"{i}. {property.county}, {property.state}")
                if 0 <= property_index < len(current_user.get_properties()):
                    calculate_roi_for_property(current_user.get_properties()[property_index])
                else:
                    print_with_lines("Invalid property number.")
            else:
                print_with_lines("Please select a user and a property first.")
        elif choice == "7":
            if current_user:
                list_properties(current_user)
            else:
                print_with_lines("Please select a user first.")
        elif choice == "8":
            break
        else:
            print_with_lines("Invalid choice. Please try again.")

if __name__ == "__main__":
    main_menu()



Welcome to the Rental Property ROI Calculator!
1. Add a new user
2. Select an existing user
3. Add a new property
4. Add income to a property
5. Add expense to a property
6. Calculate ROI for a property
7. List all properties
8. Exit

--------------------------------------------------
User 'John' added successfully.
--------------------------------------------------

Welcome to the Rental Property ROI Calculator!
1. Add a new user
2. Select an existing user
3. Add a new property
4. Add income to a property
5. Add expense to a property
6. Calculate ROI for a property
7. List all properties
8. Exit

--------------------------------------------------
Avaliable users:
--------------------------------------------------
John

--------------------------------------------------
User 'John' selected.
--------------------------------------------------

Welcome to the Rental Property ROI Calculator!
1. Add a new user
2. Select an existing user
3. Add a new property
4. Add income to a property
5.